In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
%matplotlib inline

In [ ]:
service='pet-bokeh-service'
port=8500
image_path='cat.jpg'

In [ ]:
requests.get(f'http://{service}:{port}/v1/models/pet-bokeh/metadata').json()

In [ ]:
img = image.img_to_array(image.load_img(image_path, target_size=(160, 160))) / 255
data = json.dumps({"signature_name": "",
   "instances": [img.tolist()]
})
headers = {"content-type": "application/json"}

In [ ]:
response = requests.post(f'http://{service}:{port}/v1/models/pet-bokeh:predict', data=data, headers=headers)

In [ ]:
plt.imshow(np.array(response.json()['predictions'])[0])